In [120]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import stats
import scipy.optimize

In [121]:
 
T = 10
r = 0.05
sigma = 0.4
S = 100
K = 50
t=0

In [122]:
def price_call(S, K, T, t, r, sigma):
    d1 = 1/(sigma*np.sqrt(T-t))*np.log(S*np.exp(r*(T-t))/K) + sigma*np.sqrt(T-t)/2
    d2 = 1/(sigma*np.sqrt(T-t))*np.log(S*np.exp(r*(T-t))/K) - sigma*np.sqrt(T-t)/2
    return S*stats.norm.cdf(d1) - np.exp(-r*(T-t))*K*stats.norm.cdf(d2)

C0 = price_call(S, K, T, 0, r, sigma)


In [123]:
C0 = price_call(S, K, T, 0, r, sigma) #current value of the firm’s equity

In [124]:
## call on call

In [125]:
k = [0.60, 0.80, 1, 1.2]
k = pd.Series(k)*C0
tu = [2,5,7,9]
tau=pd.Series(tu)
s_star = np.zeros((4,4))

In [126]:
s_star = np.zeros((4,4))
#Find s* using a minimisation function
i=0
j=0
for i in range(len(k)):
    for j in range(len(tau)):
        error = lambda X: (price_call(X, K, T, tau[j], r, sigma)-k[i])**2
        s_star[i,j] = scipy.optimize.fmin(func=error,x0=S) 

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 21
         Function evaluations: 42
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 40
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 20
         Function evaluations: 40
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 19
         Function evaluations: 38
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 19
         Function evaluations: 38
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 18
         Function evaluations: 36
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 17
         Function evaluations: 34
Optimization terminated successful

In [127]:
s_star

array([[ 71.07559204,  78.38737488,  84.766922  ,  92.25997925],
       [ 87.69226074,  94.96421814, 101.03637695, 107.65327454],
       [103.86741638, 111.05072021, 116.84333801, 122.85415649],
       [119.76531982, 126.84074402, 132.38945007, 137.98118591]])

In [145]:
tau=tau.to_numpy()

In [206]:
a_1 = (np.log(S/s_star) + (r+(sigma**2)/2)*(tau -t))/(sigma*np.sqrt(tau-t))
a_2 = a_1 - sigma*np.sqrt(tau-t)
b_1 = (np.log(S/D) + (r+sigma**2/2)*(T - t))/(sigma*np.sqrt(T-t))
b_2 = b_1 - sigma*np.sqrt(T-t)
rho = np.sqrt((tau-t)/(T-t))
#mu = zeros(2,1)

In [209]:
b_2

0.3108101365607241